# Blanklist Algorithm

When running the MITgcm in parallel, we partition the grid into tiles using the parameters `sNx` and `sNy` in `code/SIZE.h`. In doing so, some chunks in our partition will contain only land. We instruct the model to omit land tiles during computation using the `exch2` package. To do so, we provide a list of blank/land tiles in the input file `data.exch2`. This notebook demonstrates how to compute a `blanklist` given a global model's bathymetry `Depth` or land mask `hFacC/maskC`, and a partition.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ecco_v4_py as ecco
from MITgcmutils import rdmds
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'MITgcmutils'

# Pseudocode

```
--------------------------------------------------
FOR face_index = 1..5
    landmask_face = landmask_faces[face_index]
    FOR i = 1..sNx
        FOR j = 1..sNy
            tile = landmask_face[I,J]
            IF sum(tile)==0:
                append tile index to blank list
--------------------------------------------------     
```

> **_NOTE:_**  The function in this notebook is suited specfially for the 5-face global llc grid 


In [ ]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def get_blanklist(landmask_faces, sNx, sNy, plot=False):
    """
    Extracts and returns the indices of blank tiles from a set of landmask faces.

    Parameters:
    - landmask_faces (dict): A dictionary containing landmask arrays for each face.
    - sNx (int): Size of tiles along the x-axis.
    - sNy (int): Size of tiles along the y-axis.
    - plot (bool, optional): If True, generates a visual representation of the blank tiles
    
    Returns:
    - blanklist (list): List of indices corresponding to blank tiles.

    Note:
    - The function also supports an optional plotting feature to visualize the tiles and their indices.

    Example:
    landmask_faces = {1: np.array(...), 2: np.array(...), ...}
    sNx = 32
    sNy = 32
    blanklist = get_blanklist(landmask_faces, sNx, sNy, plot=True)
    ```
    """
    
    # initialize plot
    if plot:
        text_kwargs = dict(ha='center', va='center', fontsize=10, color='r')
        fig, axes = plt.subplots(5,1)
    
    # initialize vars
    blanklist=[]
    tile_count = 0
    
    # loop through 5 faces. Note face_index = 1..5
    for face_index, landmask_face in landmask_faces.items():
        
        # create nan mask for plotting
        blanksmask_face = np.nan * np.ones_like(landmask_face)
        nx,ny = landmask_face.shape
        
        # start tile_count from total of prev face
        tile_count0 = tile_count
        
        # chunk face into tiles of size (sNx, sNy)
        for i, ii in enumerate(chunker(np.arange(nx),sNx)):
            for j, jj in enumerate(chunker(np.arange(ny),sNy)):
                    tile_count += 1                
                    # get this tile, check if all land
                    tile=landmask_face[np.ix_(ii, jj)]
                    isblank = tile.sum() == 0
                    
                    if isblank:
                        tile_index = tile_count0 + j+i*int(ny/sNy)+1
                        blanklist.append(tile_index)
                        blanksmask_face[np.ix_(ii, jj)]=0

                    # plot tile number text
                    if plot:
                        ax = axes.ravel()[face_index-1]
                        ax.text(jj[int(sNx/2)], ii[int(sNy/2)], '{}'.format(tile_count), **text_kwargs)
        
        # plot landmask, blanks
        if plot:
            aa=ax.contourf(landmask_face, cmap='Greys_r')
            aa=ax.pcolor(blanksmask_face, cmap='jet')
            
            # set ticks
            major_ticks_x = np.arange(0, ny, sNy)
            minor_ticks_x = np.arange(0, ny) 
            major_ticks_y = np.arange(0, nx, sNx)
            minor_ticks_y = np.arange(0, nx) 

            ax.set_xticks(major_ticks_x )
            ax.set_xticks(minor_ticks_x , minor=True)
            ax.set_yticks(major_ticks_y)
            ax.set_yticks(minor_ticks_y, minor=True)
            ax.xaxis.set_ticklabels([])
            ax.yaxis.set_ticklabels([])
            
            ax.grid(which='minor', alpha=0.2)
            ax.grid(which='major', alpha=1)

            ax.set_title("Face {}".format(face_index))
            fig.set_size_inches(10,20)
    return blanklist

# Example 1: Global `llc90`

In [ ]:
# load land mask
# NOTE: using maskC/hFacC vs Depth/Bathymetry will give different results
grid_dir='/scratch/atnguyen/llc90/global_oce_llc90/GRID/'
hfc90 = rdmds(grid_dir+'hFacC', lev=0)
hfc90[hfc90!=0]=np.nan
# Convert to dict of 5 faces, sizes [(270,90), (270,90), (90,90), (90,270), (90,270)]
hfc90_faces = ecco.llc_compact_to_faces(hfc90, less_output=True)

# init dimensions
sNx=30
sNy=30

blanklist = get_blanklist(hfc90_faces, sNx, sNy, plot=True)

At this point, the user can copy the blanklist into `data.exch2`:

In [ ]:
print('  blankList = ', end='')
print(', '.join([str(x) for x in blanklist]), end='')
print(',')

# Example 2:  Global `llc1080`

In [ ]:
grid_dir='/scratch/atnguyen/llc1080/global/run_template/'

In [ ]:
# load depth1080 in compact form (13*nx, nx), then convert to 5-face dict
nx=1080
depth1080 = np.fromfile(grid_dir+'bathy1080_g5_r4_v2a.bin', dtype=np.dtype('>f')).reshape(13*nx,nx)

# nan-out ocean
depth1080[depth1080!=0]==np.nan
depth1080 = ecco.llc_compact_to_faces(depth1080, less_output=True)

In [ ]:
# init dimensions
sNx=120
sNy=120

# probably don't want to plot, will take a long time
blanklist_1080 = get_blanklist(depth1080, sNx, sNy, plot=False)
print(blanklist_1080)